# Tunning

**Import needed libaries**

In [9]:
from sklearn.datasets import make_classification
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

**Hyperparameter Tuning**

In [3]:
# Example modified from:
# https://chrisalbon.com/deep_learning/keras/tuning_neural_network_hyperparameters/

# Define the number of features
num_features=50

# Generate features matrix and target vector
# binary classification (two classes)
features, target = make_classification(n_samples = 10000,
                                       n_features = num_features,
                                       n_informative = 3,
                                       n_redundant = 0,
                                       n_classes = 2,
                                       weights = [.5, .5],
                                       random_state = 42)

# Verify the size of the features and target
print("Features array shape: ", features.shape)
print("Target array length: ", len(target))

Features array shape:  (10000, 50)
Target array length:  10000


In [4]:
# Function to return a compiled network
def make_network(optimizer='adam'):

    # Instantiate a sequential model
    network = models.Sequential()

    # Add an input layer (shape=number of features)
    network.add(layers.Dense(units=8, activation='relu', input_shape=(num_features,)))

    # Add a hidden layer with 8 neurons
    network.add(layers.Dense(units=8, activation='relu'))

    # Add an output layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    network.compile(loss='binary_crossentropy', # Cross-entropy
                    optimizer=optimizer, # Optimizer
                    metrics=['accuracy']) # Accuracy performance metric

    # Return compiled network
    return network

In [5]:
neural_network = KerasClassifier(build_fn=make_network, verbose=0)

C:\Users\tyler\AppData\Local\Temp/ipykernel_5180/2594605819.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  neural_network = KerasClassifier(build_fn=make_network, verbose=0)


In [6]:
# Define hyperparameter space over which to search
epochs = [10]
batches = [4]
optimizers = ['rmsprop', 'adam']

# Make a dictionary of the parameters
hyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)

grid = GridSearchCV(estimator=neural_network, cv=5, param_grid=hyperparameters)
grid_result = grid.fit(features, target)

**Get Results**

In [10]:
# Take a look at the best parameters
grid_result.best_params_

{'batch_size': 4, 'epochs': 10, 'optimizer': 'adam'}

**Hyper Parameter Tuning with Tensorboard**

In [11]:
# Define the number of features
num_features=50

# Generate features matrix and target vector
# binary classification (two classes)
features, target = make_classification(n_samples = 10000,
                                       n_features = num_features,
                                       n_informative = 3,
                                       n_redundant = 0,
                                       n_classes = 2,
                                       random_state = 42)

x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=42)

In [12]:
%load_ext tensorboard

In [13]:
# Specify the parameters and values
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([8, 16]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

# Evaluate the model using accuracy
METRIC_ACCURACY = 'accuracy'

# Write the function to create the logs
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

**Adapt TensorFlow runs to log hyperparameters and metrics**

In [14]:
# Write the function to create the model with the
# specified hyperparameter tuning
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

# Run with 1 epoch to speed things up for demo purposes
  model.fit(x_train, y_train, epochs=1) 
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

**Log an hparams summary with the hyperparameters and final accuracy:**

In [15]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

**Start runs and log them all under one parent directory**

In [16]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 8, 'dropout': 0.1, 'optimizer': 'adam'}
79/79 [==============================] - 0s 853us/step - loss: 1.1933 - accuracy: 0.70920s - loss: 1.1879 - accuracy: 0.70
--- Starting trial: run-1
{'num_units': 8, 'dropout': 0.1, 'optimizer': 'sgd'}
79/79 [==============================] - 0s 1ms/step - loss: 0.8750 - accuracy: 0.7536
--- Starting trial: run-2
{'num_units': 8, 'dropout': 0.2, 'optimizer': 'adam'}
79/79 [==============================] - 0s 942us/step - loss: 1.2532 - accuracy: 0.5600
--- Starting trial: run-3
{'num_units': 8, 'dropout': 0.2, 'optimizer': 'sgd'}
79/79 [==============================] - 0s 1ms/step - loss: 0.9334 - accuracy: 0.6712
--- Starting trial: run-4
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
79/79 [==============================] - 0s 868us/step - loss: 0.7453 - accuracy: 0.7388
--- Starting trial: run-5
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
79/79 [==============================] - 0s 89

**Visualize the results in TensorBoard's HParams plugin**

In [17]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 7176), started 0:09:55 ago. (Use '!kill 7176' to kill it.)